In [1]:
# some_file.py
import sys
# insert at 1, 0 is the script path (or '' in REPL)
sys.path.insert(1, '../')

# We import the base of the model
import boario.simulation as sim
# We also import the indicators module
from boario.indicators import Indicators
import pathlib

# We instantiate a dictionary with the parameters
# (it is also possible to use a json file)

params = {
    # The name of the working directory to use (relative to current wd)
    "input_dir": "/home/sjuhel/Nextcloud/Thesis/Workbench/Data/BoARIO-testing/",
    "output_dir": "/home/sjuhel/Nextcloud/Thesis/Workbench/Data/BoARIO-testing/",
    # The directory to use to store results (relative to storage_dir)
    # i.e. here, the model will look for files in ~/boario/storage/ and
    # store results in ~/boario/storage/results/
    "results_storage": "results",
    # This tells the model to register the evolution of the stocks
    # of every industry (the file can be quite large (2Gbytes+ for
    # a 365 days simulation with exiobase))
    "register_stocks": True,
    # Parameters of the model (we detail these in the documentation)
   "psi_param": 0.85,
   "model_time_step": 1,
   "timestep_dividing_factor": 365,
   "inventory_restoration_time": 9,
   "alpha_base": 1.0,
   "alpha_max": 1.25,
   "alpha_tau": 365,
   "rebuild_tau": 30,
   "n_timesteps": 365,
   "min_duration": (365 // 100) * 25,
   "impacted_region_base_production_toward_rebuilding": 0.001,
   "row_base_production_toward_rebuilding": 0.0,
    "mrio_params_file": "/home/sjuhel/Nextcloud/Thesis/Workbench/Data/Snakeruns/inputs/mrio_params.json"
}

# Here we define the event perturbing the simulation
event = {
    # The list of affected regions (their index in the mrio table)
    # Here we just have France
    "aff-regions": ["FR"],
    # The list of affected sectors
    # (here we specify that all sectors are impacted)
    "aff-sectors": "all",
    # The shares of the damages distributed between regions
    # (1 as we have only one region)
    "dmg-distrib-regions": [ 1 ],
    # The type of distribution of damages for the sectors
    # (more on that in the doc)
    "dmg-distrib-sectors-type": "gdp",
    # 'gdp' distribution doesn't require this parameter to be set
    "dmg-distrib-sectors": [],
    # The duration of the event (not implemented yet, so it has no effect)
    "duration": 1,
    # A name for the event (usefull when simulating multiple events)
    "name": "0",
    # The step at which the event shall occur during the simulation
    "occur": 5,
    # The quantity of damages caused by the event (in IO table monetary)
    "q_dmg":100000000,
    # The sectors mobilised to answer the rebuilding demand
    # and the share of the demand they answer
    "rebuilding-sectors": {
        "Construction (45)":0.15,
        "Manufacture of machinery and equipment n.e.c. (29)" : 0.20,
        "Manufacture of furniture; manufacturing n.e.c. (36)" : 0.20,
        "Manufacture of office machinery and computers (30)": 0.15,
    }
}

# We load the mrio table from a pickle file (created with the help of the
# pymrio module, more on that in the doc)
mrio_path = pathlib.Path(params['input_dir'])/"mrio_full.pkl"

In [2]:
# We initiate a model instance ...
model = sim.Simulation(params, mrio_path)

14:58:15 [INFO] - [simulation.py > __init__() > 57] - Initializing new simulation instance
14:58:15 [DEBUG] - [mriosystem.py > __init__() > 143] - Initiating new MrioSystem instance
14:58:15 [INFO] - [mriosystem.py > __init__() > 149] - Results storage is: /home/sjuhel/Nextcloud/Thesis/Workbench/Data/BoARIO-testing/results
14:58:15 [INFO] - [mriosystem.py > __init__() > 164] - Monetary unit is: 1000000
14:58:16 [INFO] - [simulation.py > __init__() > 129] - Initialized !


In [3]:
# ... add the list of events (just one here) to the model ...
model.read_events_from_list([event])

# ... and launch the simulation with :
model.loop()

14:58:16 [INFO] - [simulation.py > read_events_from_list() > 310] - Reading events from given list and adding them to the model
14:58:16 [INFO] - [simulation.py > loop() > 152] - Starting model loop for at most 365 steps
14:58:16 [INFO] - [simulation.py > loop() > 153] - One step is 1 day(s)
14:58:16 [INFO] - [simulation.py > loop() > 158] - Parameters : 
 {
    "input_dir": "/home/sjuhel/Nextcloud/Thesis/Workbench/Data/BoARIO-testing/",
    "output_dir": "/home/sjuhel/Nextcloud/Thesis/Workbench/Data/BoARIO-testing/",
    "results_storage": "results",
    "register_stocks": true,
    "psi_param": 0.85,
    "model_time_step": 1,
    "timestep_dividing_factor": 365,
    "inventory_restoration_time": 9,
    "alpha_base": 1.0,
    "alpha_max": 1.25,
    "alpha_tau": 365,
    "rebuild_tau": 30,
    "n_timesteps": 365,
    "min_duration": 75,
    "impacted_region_base_production_toward_rebuilding": 0.001,
    "row_base_production_toward_rebuilding": 0.0,
    "mrio_params_file": "/home/sjuhel

In [2]:
# Once the simulation is over we can compute some indicators :
indic = Indicators.from_folder(
                               pathlib.Path(params['output_dir'])/"results",
                               indexes_file=pathlib.Path(params['output_dir'])/"results"/"indexes.json"
)
indic.update_indicators()


15:06:25 [INFO] - [indicators.py > __init__() > 42] - Instanciating indicators
15:06:35 [INFO] - [indicators.py > save_dfs() > 346] - Saving computed dataframe to results folder
15:07:08 [INFO] - [indicators.py > update_indicators() > 330] - (Re)computing all indicators


In [3]:
indic.write_indicators()

15:07:13 [INFO] - [indicators.py > write_indicators() > 340] - Writing indicators to json


In [11]:
df2=indic.prod_df.copy()
prod_chg = df2 - df2.iloc[1,:]
prod_chg = prod_chg.round(6)
prod_chg_sect = prod_chg.sum()
prod_chg_sect.sort_values(ascending=False,key=abs).head(5).to_dict()

{('US',
  'Public administration and defence; compulsory social security, Education and Health (75,80,85)'): -4567447.260237001,
 ('US',
  'Financial intermediation, insurance and pension funding (65)'): -3781766.508188,
 ('US', 'Other business activities (74)'): -2849193.3932610005,
 ('CN',
  'Public administration and defence; compulsory social security, Education and Health (75,80,85)'): -2302412.5523510007,
 ('CN', 'Construction (45)'): -2224837.0083870003}